<a href="https://colab.research.google.com/github/MelihKlc/Deep-Learning/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP with TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences or text)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

# Token dediğimiz şey ya bir word ya da character oluyor. Bizim seçimimize göre

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-08-16 14:18:46--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.5’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-08-16 14:18:46 (87.8 MB/s) - ‘helper_functions.py.5’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback , plot_loss_curves , compare_historys , unzip_data , walk_through_dir , calculate_results

# Get the text data

The dataset we are going to use it NLP dataset for text samples tweets labelled as disaster or not disaster

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-08-16 14:18:53--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 142.251.167.128, 142.251.16.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.5’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2023-08-16 14:18:53 (85.4 MB/s) - ‘nlp_getting_started.zip.5’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

# Become one with the data

In [5]:
# Visualizing our text dataset

import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# If target label has 1 , it means that our sentence is about disasters

train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
# Shuffle training data because of we dont want to be overfitting

train_df_shuffled = train_df.sample(frac = 1 , random_state = 42) # Frac = 1 dememizin sebebi bütün datamızı shuffle etmek istiyoruz.


In [8]:
# What does the our test_dataset look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class?

train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples?

len(train_df) , len(test_df)

(7613, 3263)

In [11]:
# Lets visualize some random training examples.
import random
random_index = random.randint( 0 , len(train_df) - 5)  # Create random indexes not higher than the total number of samples

for row in train_df[["text" , "target"]][random_index : random_index + 5].itertuples():  # Bu satırda, train_df veri çerçevesinde rastgele seçilen 5 satırı (veya kalan kadar satır varsa) içeren bir alt veri çerçevesini seçiyoruz. Sadece "text" ve "target" sütunlarını seçiyoruz. itertuples() işlevi, seçilen satırları birer birer dönmek için kullanılır.
  _ , text , target = row
  print(f"Target : {target} " , "(real_disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n {text}")
  print("---\n")


Target : 1  (real_disaster)
Text:
 Train derailment: In Patna no news of any casualty so far - The Indian Express http://t.co/YH5VETm0YZ http://t.co/17Wgug8z0M
---

Target : 1  (real_disaster)
Text:
 After the green line derailment my concern for track that looks like this goes up a bit... @cta @CTAFails http://t.co/1uDz0NVOEH
---

Target : 1  (real_disaster)
Text:
 Very sad to learn of the derailment of 2 trains in Mp.My deepest condolences to the families who lost loved ones in this Mishap @OfficeOfRG
---

Target : 1  (real_disaster)
Text:
 Scene of the derailment.. CTA Green Line at 63rd/Prairie http://t.co/zz5UDiLrea
---

Target : 1  (real_disaster)
Text:
 Consent Order on cleanup underway at CSX derailment site - Knoxville News Sentinel http://t.co/xsZx9MWXYp http://t.co/NMFsgKf1Za
---



# Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

train_sentences , val_sentences , train_labels , val_labels = train_test_split(train_df_shuffled["text"].to_numpy() ,train_df_shuffled["target"].to_numpy() , test_size=0.1 , random_state = 42)

In [13]:
len(train_sentences) , len(val_sentences) , len(train_labels) , len(val_labels)

(6851, 762, 6851, 762)

In [14]:
 # Check the first 10 samples

 train_sentences[ : 10] , train_labels[ : 10]  # Train sentence içindeki cümleler train_labels taki outputlarla eşleşiyor

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

When dealing with a text problem , one of the first things you will have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this namely:
* Tokenization - Direct mapping of token (a token could be a word or a character) to number.
* Embedding - Create a matrix of feature vector for each token (the size of the feature vector can be and this embedding can be learn)

# Text vectorization (tokenization , Converting text into numbers)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#ngram substringleri birleştiriyor. Mesela elimizde 'Imagine getting flattened by Kurt Zouma' cümlesi var. ngram = 2 dersek , "Image getting" , "flattened by" , "Kurt Zouma"  gibi ikişer ikişer substringleri birleştiriyor.

# Default values of TextVectorization
text_vectorizer = TextVectorization(max_tokens = 10000 , # How many words in the vocabulary ( automatically add <00V>)
                                                        standardize = "lower_and_strip_punctuation" ,  # Bütün harfleri küçük yapıp noktalama işaretlerini siliyor çünkü bunlar disaster olup olmadıgını anlamamıza yardım eden şeyler değil.
                                                        split = "whitespace"  , # Tokenlar arasında boşluk bırakmak için.
                                                        ngrams = None , # Create a groups of n-word
                                                        output_mode = "int" , # How to map tokens to number
                                                        output_sequence_length = None , # How long do you want your sequences to be. (None atadıgımızda en uzun tokeni alıp bütün tokenlere o lengthi veriyor. Kısa olan sentences ların sonu 0 larla dolduruluyor.)
                                                        pad_to_max_tokens = True
                                    )

In [16]:
train_sentences[0].split()  # Split bu işe yarıyor.

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [17]:
# Find the average number of tokens (words) in the training tweets
# We could just pad them to the longest , but to keep our data small , we will just find the average lenght and then make sure every tweet gets turned into numbers but those list of numbers is the same size as the average throughout every sample.

round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [18]:
# Setup text vectorization variables.

max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 # Max length our sequences will be (e.g. How many words from a Tweet does model see?). Mesela tweet 30 kelime uzunlugunda ama biz bunu 15 atadıgımız icin modelimiz ilk 15 kelimesini görecek

# Uptading our textvectorizer

text_vectorizer = TextVectorization(max_tokens = max_vocab_length ,  # Bizim bütün tweetlerimizden 10000 tane alacak bu 10000 most common lardan oluşacak
                                    output_mode = "int" ,
                                    output_sequence_length = max_length
                                    )


In [19]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [20]:
# Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)  # CNN deki fit işlemi gibi artık bütün train sentenceslarımıza text_vectorizerde ne yapılmasını istediysek uygulanacak.

In [21]:
# We are trying to do here is map the word data that we have to numerical form

In [22]:
# Create a sample sentence and tokenize it

sample_sentence = "There's a flood in my street!"

text_vectorizer([sample_sentence])   # Text vectorizer ı biz liste şeklinde oluşturdugumuz icin girdigimiz örnekleri [ ] içinde yazıyoruz.
# Burda shape in 0. indexi sequence i veriyor. 1 tam cümle girdiğimiz için sequence 1.
# Shape in 1. indexi de length veriyor. Max olarak 15 dediğimiz için buna adapt oluyo bizim sample ımız.

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
# Choose a random sentence from our train_sentences data

import random
random_sentences = random.choice(train_sentences)

print(f"Original sentence : \n {random_sentences} \n\n  Vectorized form of our random sentence: \n")
text_vectorizer([sample_sentence])

Original sentence : 
 @Tellyfckngo @JayCootchi nah you hit homie wit the fatality and then son'd him wit the babality. Cold. Cold as fuck lmfaoooo. 

  Vectorized form of our random sentence: 



<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()  # Get all of the unique words in our training data
top_5_words = words_in_vocab[ : 5 ] # Get the most common words in our data
bottom_5_words = words_in_vocab[ -5 :  ] # Get the least common words

print(f"Number of words in vocab : {len(words_in_vocab)} \n")
print(f"Most common words : {top_5_words} \n")
print(f"Least common words  : {bottom_5_words} \n")

Number of words in vocab : 10000 

Most common words : ['', '[UNK]', 'the', 'a', 'in'] 

Least common words  : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'] 



# Creating an Embedding using an Embedding Layer

To make our embedding we are going to use TensorFlow embedding layer.

The parameters we care most about for our embedding layers

* 'input_dim' : the size of our vocabulary
* 'output_dim' : the size of the output embedding vector , for example , a value of 100 would mean each token gets represented by a vector 100 long.
* 'input_length' : lenght of the sequences being passed to the embedding layer.

# **Note** : Neural networks 8 e bölünebilen sayılarla çok iyi çalışıyor

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding( input_dim = max_vocab_length ,  # set input shape
                             output_dim = 128 , # Output shape. Her token in the format of 128 long vector
                              input_length = max_length  # How long is each input
                              )

In [26]:
# Get a random sentence from our training data

random_sentence = random.choice(train_sentences)
print(f"Original text : \n {random_sentence}\n")
print("Embedded form")
sample_embed = embedding(text_vectorizer([sample_sentence]))  # Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed

Original text : 
 Damn...was wondering where my drone ended up after the freak windstorm...?? https://t.co/dHgGxo7Mcc

Embedded form


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03995016, -0.02617685,  0.01450134, ..., -0.01942835,
         -0.04916811,  0.02832599],
        [ 0.04135284, -0.03055332,  0.0145925 , ..., -0.04591091,
          0.02767311, -0.0374121 ],
        [-0.02926223,  0.04855502,  0.00105126, ...,  0.04940287,
          0.03327012,  0.01327053],
        ...,
        [ 0.03541305, -0.01300358,  0.04801238, ..., -0.04914395,
          0.02625049, -0.01532371],
        [ 0.03541305, -0.01300358,  0.04801238, ..., -0.04914395,
          0.02625049, -0.01532371],
        [ 0.03541305, -0.01300358,  0.04801238, ..., -0.04914395,
          0.02625049, -0.01532371]]], dtype=float32)>

In [27]:
# Check out a single token's embedding

sample_embed[0][0] , sample_embed[0][0].shape , sample_embed

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03995016, -0.02617685,  0.01450134, -0.01314858,  0.00793686,
         0.03568694, -0.00762456,  0.01633609,  0.036788  ,  0.01904053,
         0.0061638 , -0.0425276 , -0.02676259, -0.02047646,  0.04030004,
         0.001499  ,  0.01586526, -0.00799207,  0.0130068 , -0.04497341,
        -0.02335066, -0.04388143, -0.01203499, -0.04793221,  0.0039552 ,
         0.0075004 , -0.03918009,  0.03994069,  0.01945826,  0.00166823,
        -0.00377128, -0.00837677,  0.04003662,  0.00133556,  0.02693087,
         0.03131041,  0.00149997, -0.01514667,  0.04928024, -0.02466735,
         0.03015083,  0.02463149, -0.03539721, -0.02905909,  0.02795844,
         0.03580518,  0.03094686, -0.03607934,  0.03912416, -0.04472167,
         0.01142649,  0.0430688 ,  0.0047547 , -0.04061315,  0.0309623 ,
         0.00752572,  0.02713743,  0.02081133,  0.03072867, -0.00939314,
        -0.04220345, -0.03646002,  0.02953063, -0.02192189,  0.0189752 ,
  

# Modelling a text dataset (running a series of experimentation)

Now we have got a way to turn our text sequences into numbers , it is time to start building a series of modelling experiments.

We will start with a baseline and move on from there

* Model 0 : Naive Bayes (baseline)
* Model 1 : Feed-forward neural network
* Model 2 : LSTM model (RNN)
* Model 3 : GRU model (RNN)
* Model 4 : Bidirectional LSTM (RNN)
* Model 5 : 1D Convolutional neural network (CNN)
* Model 6 : TensorFlow Hub pre-trained Feature extractor (using transfer learning for NLP)
* Model 7 : Same as model 6 but with %10 of training data

How are we going to approach all of this ?

Use standart steps in modelling with tensorflow

* Create a model
* Compile o model
* Fit the model
* Evaluate the model

## Model 0 : Getting a baseline

As with all machine learning modelling experiments , it is important to create a baseline model so you have got a benchmark for future experiments to build upon

To create our baseline, We will use sklearn Multinomial naive bayes using the TF-IDF formula to convert our words to numbers.

> **Note:** It is common practice to use non deep learning algorithms as a baseline because of their speed then later using deep learning to see if you can improve upon them

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline(
    [
     ("tfidf" , TfidfVectorizer())   , # Convert words to numbers using tfidf
     ("clf" , MultinomialNB() ) # Model to text

    ]

)

# Fit the pipeline to the training data
model_0.fit(train_sentences , train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our model
baseline_score = model_0.score(val_sentences , val_labels)  # In sklearn we use score instead of evaluate
baseline_score
# Bu bize accuracy veriyor.


0.7926509186351706

In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[ : 20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [31]:
val_labels[ : 20]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

# Creating an evaluation function for our model experiment

We could evaluate all of our model's predictions with different metrics every time , however , this will be cumbersome(hantal) and could easliy be fixed with a function

Lets create one to compare our model's predictions with the truth labels using following metrices

* Accuracy
* Precision
* Recall
* F1-score

In [32]:
# Function to evaluate : accuracy , precision , recall , f1-score
from sklearn.metrics import accuracy_score , precision_recall_fscore_support

def calculate_results(y_true , y_pred):
  """
  Calculates model accuracy , precision , recall and f1 score of a binary classification model
  """

  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true , y_pred)

  # Calculate model precision , recall and f1 score using "weighted" average

  model_precision , model_recall , model_f1 , _ = precision_recall_fscore_support(y_true , y_pred , average = "weighted")
  model_results = {
      "accuracy": model_accuracy ,
      "precision": model_precision ,
      "recall": model_recall ,
      "f1-score": model_f1
  }
  return model_results

In [33]:
# Get model results

baseline_results = calculate_results(y_true = val_labels ,
                                     y_pred = baseline_preds
                                     )
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

# Model 1 : A simple dense model

In [34]:
# Create a tensorboard callback (need to create a new one for each model)

from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
save_dir = "model_logs"

In [35]:
# Build a model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,) , dtype = tf.string)  # shape = (1 , ) çünkü it is going to see one sequence at a time
x = text_vectorizer(inputs)  # Turn the input text into numbers
x = embedding(x) # Create an embedding of the numberized inputs.
x = layers.GlobalAveragePooling1D()(x) # Lower the dimensionality of the embedding. Eğer pooling layer eklemezsek her token için ayrı ayrı predict yapıyor. Pooling layer tek bir uzun vectore çeviriyor.
outputs = layers.Dense( 1 , activation = "sigmoid")(x)  # Create an output layer , We have binary outputs so we use 1 hidden unit and activation will be sigmoid

model_1 = tf.keras.Model(inputs , outputs , name = "model_1_dense")

In [36]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [37]:
# Compile a model
model_1.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]
                )

# Fit the model

hist_1 = model_1.fit(x = train_sentences ,
                     y = train_labels ,
                     epochs = 5 ,
                     validation_data = (val_sentences , val_labels)  ,
                     callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_1_dense")])   # validation datayı tuple formatta yazmamız gerekiyor.

Saving TensorBoard log files to: model_logs/model_1_dense/20230816-141858
Epoch 1/5
215/215 [==============================] - 12s 48ms/step - loss: 0.6119 - accuracy: 0.6865 - val_loss: 0.5372 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 9s 42ms/step - loss: 0.4429 - accuracy: 0.8183 - val_loss: 0.4709 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 9s 42ms/step - loss: 0.3476 - accuracy: 0.8587 - val_loss: 0.4572 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 9s 42ms/step - loss: 0.2860 - accuracy: 0.8899 - val_loss: 0.4620 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 8s 39ms/step - loss: 0.2385 - accuracy: 0.9120 - val_loss: 0.4801 - val_accuracy: 0.7848


In [38]:
# Check the results
model_1.evaluate(val_sentences , val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4801 - accuracy: 0.7848


[0.4800863265991211, 0.7847769260406494]

In [39]:
# Make some predictions and evaluate those
model_1_preds = model_1.predict(val_sentences)
model_1_preds.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [40]:
# Look at a single prediction
model_1_preds[0]

array([0.35682356], dtype=float32)

In [41]:
 val_labels  # Şu an val labels ile predlerimizi karşılaştıramıyoruz çünkü biz apple to apple gibi aynı şeyleri karşılaştırmamız gerek. Zaten binary classificationdan bildigimiz üzere şu an predictlerimiz probabilities şeklinde yani 0 la 1 arasında değerler alıyor. 0.5 in üstündeki değerleri 1 olarak alıcaz altındaki değerleri ise 0 olarak alıcaz. Bu şekilde pred değerlerimizle ground truth değerleri karşılaştırabiliriz.


array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [42]:
# Convert model prediction probabilities to label format

model_1_pred = tf.squeeze(tf.round(model_1_preds))
model_1_pred[ : 10] , model_1_pred.shape

(<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>,
 TensorShape([762]))

In [43]:
# Calculate our model_1 results

model_1_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_1_pred
                                    )
model_1_results

{'accuracy': 0.7847769028871391,
 'precision': 0.789165199286798,
 'recall': 0.7847769028871391,
 'f1-score': 0.7818959205825942}

In [44]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

# Looks like our baseline is outperforming our first deep learning model

# Embedding layer en başta random değerler alıyor. Sonrasında her epochta internal weights ini uptade for each token.  

# Visualizing learned embeddings

In [45]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [46]:
# Get the weight matrix of embedding layer
# These are the numerical representation of each token in our training data
# Getlayer kullanırken summary içindeki ismi kullanıyoruz.
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [47]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


# Now we have got the embedding matrix our model has learned to represent our tokens , lets visualize this

Handy tool form tensoflow : Tensorflow Projector

https://www.tensorflow.org/text/guide/word_embeddings

In [48]:
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [49]:
# To upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

https://www.youtube.com/watch?v=SEnXr6v2ifU


# Model 2: LSTM

LSTM = Long Short Term Memory

Our structure of an RNN typically looks like this:

'''
Input(text) -> Tokenize -> Embedding -> Layers (RNN's / dense) -> Output(label probability)
'''

In [52]:
# Create an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape = (1 ,)  , dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(64 , return_sequences = True)(x)  # When you are stacking RNN cells together , you need to set return_sequences = True. Yani eğer birden fazla lstm layer kullanacaksak son lstm layer hariç true atamamız lazım.
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64 , activation = "relu")(x)
outputs = layers.Dense(1 , activation = "sigmoid")(x)

model_2 = tf.keras.Model(inputs , outputs)


(None, 15, 128)
(None, 15, 64)
(None, 64)


In [53]:
model_2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_2 (LSTM)               (None, 15, 64)            49408     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                           

In [54]:
# Compile a model

model_2.compile(loss = ["binary_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]

                )

hist_2 = model_2.fit(x = train_sentences ,
                     y = train_labels ,
                     epochs = 5 ,
                     validation_data = (val_sentences , val_labels)  ,
                     callbacks = [create_tensorboard_callback(dir_name = save_dir , experiment_name = "model_2_dense")])

Saving TensorBoard log files to: model_logs/model_2_dense/20230816-151150
Epoch 1/5
215/215 [==============================] - 16s 50ms/step - loss: 0.2273 - accuracy: 0.9226 - val_loss: 0.5413 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 9s 40ms/step - loss: 0.1599 - accuracy: 0.9399 - val_loss: 0.6988 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 9s 43ms/step - loss: 0.1304 - accuracy: 0.9510 - val_loss: 0.7053 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 9s 44ms/step - loss: 0.1070 - accuracy: 0.9602 - val_loss: 1.0449 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 11s 51ms/step - loss: 0.0849 - accuracy: 0.9653 - val_loss: 1.0295 - val_accuracy: 0.7769


In [55]:
model_2.evaluate(val_sentences , val_labels)

24/24 [==============================] - 1s 17ms/step - loss: 1.0295 - accuracy: 0.7769


[1.0294981002807617, 0.7769029140472412]

In [57]:
model_2_preds = model_2.predict(val_sentences)

24/24 [==============================] - 0s 12ms/step


In [59]:
model_2_pred = tf.squeeze(tf.round(model_2_preds))
model_2_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0

In [60]:
model_2_results = calculate_results(y_true = val_labels ,
                                    y_pred = model_2_pred
                                    )
model_2_results

{'accuracy': 0.7769028871391076,
 'precision': 0.7798819103655997,
 'recall': 0.7769028871391076,
 'f1-score': 0.7743498017353971}